# PyDrive


# Tweet Collection

In [ ]:
#collecting from query search
import tweepy
from google.colab import drive
import json 
from datetime import date
from datetime import datetime
import time

drive.mount('[your drive]')

path= '[path]'

#Twitter Authentication 
auth = tweepy.OAuthHandler("Tweepy Authenticator")
auth.set_access_token("Access Token")

api = tweepy.API(auth, wait_on_rate_limit=True)

search_query = ["#letfansin"]

data = []

def search_tweets(search_term):
  counter = 0
  for tweet in tweepy.Cursor(api.search, q='\"{}\" -filter:retweets'.format(search_term), lang='en', tweet_mode='extended', result_type='recent').items(9000):
    tweet_details = {}
    tweet_details['name'] = tweet.user.screen_name
    tweet_details['tweet'] = tweet.full_text
    tweet_details['retweets'] = tweet.retweet_count
    tweet_details['location'] = tweet.user.location
    tweet_details['created'] = tweet.created_at.strftime("%d-%b-%Y")
    tweet_details['followers'] = tweet.user.followers_count
    tweet_details['is_user_verified'] = tweet.user.verified

    data.append(tweet_details)

    counter += 1
    if counter == 6000:
      break
    else:
        pass

  with open("data.json", 'w') as outfile:
    json.dump(data, outfile)

if __name__ == "__main__":
  print("start")
  for search_term in search_query:
    search_tweets(search_term)
  print('done')



In [ ]:
import pandas as pd
df_json = pd.read_json("/content/drive/MyDrive/CSE5095/dataset_epoch2.json")
df_json.to_excel("/content/drive/MyDrive/CSE5095/dataset_epoch2.xlsx")


In [ ]:
#collecting replies from specific tweet
import tweepy
from google.colab import drive
import json 
from datetime import date
from datetime import datetime
import time

drive.mount('[insert drive]')

path= '[insert path]'

#Twitter Authentication 
auth = tweepy.OAuthHandler("[inset tweepy authenticator]")
auth.set_access_token("[insert access token]")

api = tweepy.API(auth, wait_on_rate_limit=True)

data=[]
for full_tweets in tweepy.Cursor(api.user_timeline,screen_name=name,timeout=999999).items(10):
  for tweet in tweepy.Cursor(api.search,'to:{} filter:replies'.format(name),result_type='recent',timeout=999999, lang='en', tweet_mode='extended').items(1000):
    tweet_details = {}
    tweet_details['name'] = tweet.user.screen_name
    tweet_details['tweet'] = tweet.full_text
    tweet_details['retweets'] = tweet.retweet_count
    tweet_details['location'] = tweet.user.location
    tweet_details['created'] = tweet.created_at.strftime("%d-%b-%Y")
    tweet_details['followers'] = tweet.user.followers_count
    tweet_details['is_user_verified'] = tweet.user.verified
    if hasattr(tweet, 'in_reply_to_status_id_str'):
      if (tweet.in_reply_to_status_id_str==tweet_id):
        data.append(tweet_details)   

    with open("data.json", 'w') as outfile:
      json.dump(data, outfile)

#Negative Emotions


In [ ]:
#!pip install emot

import pandas as pd
import numpy as np
import re
import nltk
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
import emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import matplotlib.pyplot as plt
%matplotlib inline

dataSet = pd.read_csv("/content/drive/Shareddrives/Social Media Mining/test_2.csv")

dataSet = dataSet[dataSet['neg_emo'].notna()]

features = dataSet.iloc[1:,2].values
labels = dataSet.iloc[1:,10].values

processed_features = []

def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def remove_stopwords(text):
  stopWords = stopwords.words('english')
  not_stopword = [word for word in text if not word in stopWords]
  return not_stopword

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)


for sentence in range(0, len(features)):

    #Convert Emojis into text
    processed_feature = convert_emojis(str(features[sentence]))

    processed_feature = processed_feature.replace("_", " ")

    #remove links
    processed_feature = re.sub(r"http\S+", "", processed_feature)

    processed_feature = re.sub(r'\W', ' ', processed_feature)

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_feature = lemmatize_sentence(processed_feature)

    processed_features.append(processed_feature)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 


my_additional_stop_words = (['heart','letfansin', 'club', 'face', 'people', 'like', 'goal', 'circle' , 'circlewhite', 'circlered', 'northern', 'ireland', 'league', 'team', 'floor', 'laugh', 'talksportdrive', 'klfcofficial' , 'skin', 'tone', 'game', 'engwal', 'male', 'female', 'cc', 'soccer', 'fawales', 'fa', 'ball'])

vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=4, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words))
processed_features_tfidf = vectorizer.fit_transform(processed_features).toarray()

from sklearn.feature_selection import chi2
import numpy as np

#oversample using SMOTE
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

strategy = {'sad':600, 'ang':600, 'fea':200}
oversample = SMOTE(sampling_strategy= strategy)
undersample= TomekLinks("not minority")

processed_features_tfidf, labels = oversample.fit_resample(processed_features_tfidf, labels)
processed_features_tfidf, labels = undersample.fit_resample(processed_features_tfidf, labels)

#train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(processed_features_tfidf, labels, test_size=0.2, random_state=8)


#import cross_val, metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#random forest
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')


print("Random Forest Scores:\n")
print(classification_report(y_test,predictions))
#print("Accuracy Score:\n")
print("5-fold Cross Validation Accuracy Scores:\n")
print(accuracy_score(y_test, predictions))
#print("5-fold Cross Validation Accuracy Scores:\n", cross_val.mean(), "\n")

#Logistic Regression
from sklearn.linear_model import LogisticRegression

text_classifier = LogisticRegression(random_state=166)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')


print("Logistic Regression Scores:\n")
print(classification_report(y_test, predictions))
#print("Accuracy Score:\n")
print("5-fold Cross Validation Accuracy Scores:\n")
print(accuracy_score(y_test, predictions))
#print("5-fold Cross Validation Accuracy Scores:\n", cross_val.mean(), "\n")

#LinearSVC
from sklearn.svm import LinearSVC

text_classifier = LinearSVC()
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')

print("LinearSVC Scores:\n")
print(classification_report(y_test, predictions))
#print("Accuracy Score:\n")
print("5-fold Cross Validation Accuracy Scores:\n")
print(accuracy_score(y_test, predictions))
#print("5-fold Cross Validation Accuracy Scores:\n", cross_val.mean(), "\n")




/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Random Forest Scores:

              precision    recall  f1-score   support

         ang       0.81      0.63      0.71       117
         dis       0.70      0.90      0.79       213
         fea       1.00      0.84      0.91        31
         sad       0.84      0.65      0.73       124

    accuracy                           0.77       485
   macro avg       0.84      0.76      0.79       485
weighted avg       0.78      0.77      0.76       485

5-fold Cross Validation Accuracy Scores:

0.7670103092783506
Logistic Regression Scores:

              precision    recall  f1-score   support

         ang       0.76      0.50      0.60       117
         dis       0.67      0.91      0.77       213
         fea       0.92      0.71      0.80        31
         sad       0.88      0.68      0.77       124

    accuracy                           0.74       485
   macro avg       0.81      0.70      0.73       485
weighted avg       0.76      0.74      0.73       485

5-fold Cross Vali

In [ ]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')

print("Random Forest Cross Validation Score:\n", cross_val.mean())
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions), "\n")

#multinomialNB
from sklearn.naive_bayes import MultinomialNB

text_classifier = MultinomialNB()
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')

print("mNB Cross Validation Score:\n", cross_val.mean())
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions), "\n")

#Logistic Regression
from sklearn.linear_model import LogisticRegression

text_classifier = LogisticRegression(random_state=0)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')


print("Logistic Regression Cross Validation Score:\n", cross_val.mean())
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions), "\n")

#LinearSVC
from sklearn.svm import LinearSVC

text_classifier = LinearSVC()
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')

print("LinearSVC Cross Validation Score:\n", cross_val.mean())
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions), "\n")

In [ ]:
#train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(processed_features_tfidf, labels, test_size=0.2, random_state=0)


#import cross_val, metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#random forest
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')

#print("Random Forest Cross Validation Score:\n", cross_val.mean())
print("Random Forest Cross Validation Accuracy Scores:\n")
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions), "\n")

#Logistic Regression
from sklearn.linear_model import LogisticRegression

text_classifier = LogisticRegression(random_state=0)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')


#print("Logistic Regression Cross Validation Score:\n", cross_val.mean())
print("Logistic Regression Cross Validation Accuracy Scores:\n")
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions), "\n")

#LinearSVC
from sklearn.svm import LinearSVC

text_classifier = LinearSVC()
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

cross_val = cross_val_score(text_classifier, processed_features_tfidf, labels, scoring='accuracy')

#print("LinearSVC Cross Validation Score:\n", cross_val.mean())
print("LinearSVC Cross Validation Scores:\n")
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions), "\n")